<a href="https://colab.research.google.com/github/GaohuiXu/llm-cookbook/blob/main/content/%E5%BF%85%E4%BF%AE%E4%B8%80-Prompt%20Engineering%20For%20Developers/3.%20%E8%BF%AD%E4%BB%A3%E4%BC%98%E5%8C%96%20Iterative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第三章 迭代优化

当使用 LLM 构建应用程序时，实践层面上很难*第一次尝试*就成功获得适合最终应用的 Prompt。但这并不重要，只要您有一个好的迭代过程来不断改进您的 Prompt，那么您就能够得到一个适合任务的 Prompt。虽然相比训练机器学习模型，在  Prompt 方面一次成功的几率可能会高一些，但正如上所说， Prompt 是否一次完善并不重要。最重要的是**层层迭代**为您的应用程序找到有效  Prompt 的过程。

因此在本章中，我们将以产品说明书中生成营销文案为例，来展示一些流程框架，并提示您思考如何层层迭代地分析和完善您的 Prompt。

在吴恩达（Andrew Ng，原教程作者）的机器学习课程中展示过一张图表，说明了机器学习开发的流程。通常是先有一个想法，然后再用以下流程实现：编写代码，获取数据，训练模型，获得实验结果。然后您可以查看结果，分析误差与错误，找出适用领域，甚至可以更改您对具体问题的具体思路或解决方法。此后再次更改实现，并运行另一个实验等，反复迭代，最终获得有效的机器学习模型。在编写基于 LLM 的应用程序的 Prompt 时，流程可能非常相似。您产生了关于要完成的任务的想法后，可以尝试编写第一个 Prompt ，注意要满足上一章说过的两个原则：**清晰明确，并且给系统足够的时间思考**。然后您可以运行并查看结果。如果第一次效果不好，那么迭代的过程就是找出为什么指令不够清晰或为什么没有给算法足够的时间思考，以便改进想法、改进  Prompt 等等，循环多次，直到找到适合您的应用程序的 Prompt。

很难有适用于世间万物的所谓“最佳  Prompt ”，更好的方法是找到有效的迭代过程，以便您可以快速地找到一个适合您的应用程序的  Prompt 。


<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一环境配置" data-toc-modified-id="一、环境配置">一、环境配置</a></span></li>
        <li>
            <span><a href="#二任务从产品说明书生成一份营销产品描述" data-toc-modified-id="二、任务——从产品说明书生成一份营销产品描述">二、任务——从产品说明书生成一份营销产品描述</a></span>
            <ul class="toc-item">
                <li><span><a href="#21-问题一生成文本太长" data-toc-modified-id="2.1 问题一：生成文本太长">2.1 问题一：生成文本太长</a></span></li>
                <li><span><a href="#22-问题二抓错文本细节" data-toc-modified-id="2.2 问题二：抓错文本细节">2.2 问题二：抓错文本细节</a></span></li>
                <li><span><a href="#23-问题三添加表格描述" data-toc-modified-id="2.3 问题三：添加表格描述">2.3 问题三：添加表格描述</a></span></li>
            </ul>
        </li>
    </ul>
</div>

In [1]:
!pip install openai==0.28

## 一、环境配置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [2]:
import openai
# 导入第三方库

openai.api_key  = "sk-de071700bc094713b4567d255658606f"
openai.api_base = "https://api.deepseek.com/v1"
# 设置 API_KEY, 请替换成您自己的 API_KEY


In [3]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="deepseek-reasoner"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


## 二、任务——从产品说明书生成一份营销产品描述

给定一份椅子的资料页。描述说它属于*中世纪灵感*系列，产自意大利，并介绍了材料、构造、尺寸、可选配件等参数。假设您想要使用这份说明书帮助营销团队为电商平台撰写营销描述稿：

In [4]:
# 示例：产品说明书
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [5]:
#   Prompt ：基于说明书生成营销描述
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


**Mid-Century Modern Office Chair – Elegance Meets Functionality**  

Elevate your workspace with this exquisitely crafted mid-century inspired office chair, part of a sophisticated furniture collection designed to harmonize with desks, bookcases, and filing cabinets. Blending timeless aesthetics with modern comfort, this chair is perfect for both home offices and professional settings, offering versatility and durability certified for contract use.  

**Customizable Design**  
Choose from a range of shell colors and premium base finishes (stainless steel, matte black, gloss white, or chrome) to match your décor. Opt for partial upholstery (SWC-100) or full upholstery (SWC-110) in 10 luxurious fabric or 6 rich leather options. Prefer armrests? Select armless simplicity or 8-position adjustable PU armrests for tailored support.  

**Engineered for Comfort & Durability**  
- **Smooth Adjustability**: Pneumatic seat height adjustment ensures effortless transitions between sitting and stan

In [6]:
# 示例：产品说明书
fact_sheet_chair = """
概述

    美丽的中世纪风格办公家具系列的一部分，包括文件柜、办公桌、书柜、会议桌等。
    多种外壳颜色和底座涂层可选。
    可选塑料前后靠背装饰（SWC-100）或10种面料和6种皮革的全面装饰（SWC-110）。
    底座涂层选项为：不锈钢、哑光黑色、光泽白色或铬。
    椅子可带或不带扶手。
    适用于家庭或商业场所。
    符合合同使用资格。

结构

    五个轮子的塑料涂层铝底座。
    气动椅子调节，方便升降。

尺寸

    宽度53厘米|20.87英寸
    深度51厘米|20.08英寸
    高度80厘米|31.50英寸
    座椅高度44厘米|17.32英寸
    座椅深度41厘米|16.14英寸

选项

    软地板或硬地板滚轮选项。
    两种座椅泡沫密度可选：中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。
    无扶手或8个位置PU扶手。

材料
外壳底座滑动件

    改性尼龙PA6/PA66涂层的铸铝。
    外壳厚度：10毫米。
    座椅
    HD36泡沫

原产国

    意大利
"""

In [7]:
#   Prompt ：基于说明书创建营销描述
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的营销描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

技术说明: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


**经典重生，优雅永不过时——SWC系列中世纪风办公椅**  

在复古美与现代功能的交汇处，SWC系列办公椅以**20世纪中叶设计精髓**重新定义办公美学。作为中世纪风格家具套系的核心单品，这把椅子不仅是工具，更是空间的艺术品——无论置于居家书房、创意工作室，还是高端商务场所，它都将成为目光的焦点。  

---  

### 🔍 核心亮点  

**1. 量身定制的美学表达**  
- **12色外壳+4种涂层底座**：从哑光黑的深邃到铬金属的冷冽，底座与外壳色彩自由碰撞，适配任何空间调性。  
- **双重装饰升级**：  
  ✔️ **SWC-100** 简约款：轻量级塑料靠背装饰，线条利落。  
  ✔️ **SWC-110** 尊享款：16种高级面料与皮革（从亲肤棉麻到意大利头层牛皮），触感与视觉的双重奢华。  

**2. 工程师级别的舒适哲学**  
- **动态气压升降系统**：一指轻触即可精准调节座椅高度，适应不同体型需求。  
- **双密度HD36泡沫座垫**：  
  ➤ 中等密度（1.8磅/立方英尺）：日常办公的均衡支撑。  
  ➤ 高密度（2.8磅/立方英尺）：久坐不塌陷的终极解决方案。  
- **8段位PU扶手**：从完全收纳到贴合肘部的黄金角度，满足专注办公或休闲后仰的多场景需求。  

**3. 细节铸就品质**  
- **五轮铝制悬浮底盘**：塑料涂层消音轮可选配软/硬地板专用款，静音滑动中尽显意大利精工——哪怕在百年老宅的木地板上，也绝不留下划痕。  
- **10mm加厚改性尼龙外壳**：轻量化设计与军工级耐磨损性的完美平衡，承重可达300小时连续压力测试。  

---  

### 📐 智慧尺寸，隐形收纳  
53cm（宽）×51cm（深）的紧凑底座，却能托起44cm符合人体工学的座高（相当于膝盖自然弯曲的黄金比例）。当它静静停驻在书桌前时，所占空间仅相当于一块中号地毯，小户型也能优雅驾驭。  

---  

**🇮🇹 意大利弗留利大区制造**  
每一把SWC椅子的金属部件上都镌刻着威尼托工匠的签名钢印——这不仅是家具，更是可传承的设计遗产。  

---  

**🛋️ 现在行动，解锁两种特权：**  
✅ 商用级认证：符合ASTM国际耐久性标准，酒店、共享办公空间批量采购享专属折扣。  
✅ 十年金属结构

## 2.1 问题一：生成文本太长

它似乎很好地完成了要求，即从技术说明书开始编写产品描述，介绍了一个精致的中世纪风格办公椅。但是当我看到这个时，我会觉得这个太长了。

所以在上述过程中，我产生想法后写了一个  Prompt ，并得到了结果，但是我对它不是很满意，因为它太长了。所以我澄清我的  Prompt ，要求它限制生成文本长度，要求最多使用50个字。


In [10]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Elevate your workspace with this elegant medieval-inspired office chair, part of a sophisticated collection. Choose from versatile colors, stainless steel or chrome bases, and fabric/leather upholstery options. Features swivel casters, pneumatic height adjustment, and optional PU armrests. Crafted with durable nylon-coated aluminum and HD36 foam. Made in Italy. Perfect for home or commercial use.


提取回答并根据空格拆分，答案为54个字，较好地完成了设计要求。

In [ ]:
lst = response.split()
print(len(lst))

54


In [15]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

使用最多50个汉字。

技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


中世纪风格办公椅，可选多色外壳/底座涂层，皮革或织物饰面。五轮气动升降设计，适配软硬地面。高密度泡沫座垫，8向扶手调节，意大利制造。（50字）


In [16]:
# 由于中文需要分词，此处直接计算整体长度
len(response)

71

LLM在能堪堪胜任严格的字数限制，但实现得并不精确。此例中，英文输出要求控制在50个词，但有时会输出60或65个单词的内容，但这也还算合理。原因是 LLM 使用分词器（tokenizer）解释文本，但它们往往在计算字符方面表现一般般。有很多不同的方法来尝试控制您得到的输出的长度（如若干句话/词/个汉字/个字母 (characters) 等）。

## 2.2 问题二：抓错文本细节

我们继续完善这段推广词，会发现的第二个问题是，这个网站并不是直接向消费者销售，它实际上面向的是家具零售商，他们会更关心椅子的技术细节和材料。在这种情况下，您可以继续修改这个  Prompt ，让它更精确地描述椅子的技术细节。

解决方法：要求它专注于与目标受众相关的方面。

In [ ]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our beautiful medieval-style office furniture collection, including file cabinets, desks, bookcases, and conference tables. Available in multiple shell colors and base coatings, with optional plastic or fabric/leather upholstery. Features a plastic-coated aluminum base with five wheels and pneumatic chair adjustment. Suitable for home or commercial use and made with high-quality materials, including cast aluminum with a modified nylon coating and HD36 foam. Made in Italy.


In [19]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

注意，具体的材料必须要告知零售商。

使用最多100个汉字。

技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

中世纪风格办公椅，外壳采用改性尼龙PA6/PA66涂层铸铝（10mm厚），五轮塑料覆铝底座可选不锈钢/哑光黑/亮白/铬涂层。座椅配备HD36泡沫（1.8/2.8磅密度可选），支持气动升降。提供PU八向扶手、尼龙/全包饰面（SWC-100塑料或SWC-110十布六皮），软硬地板滚轮配置。意大利制造，符合商用合约标准。


可见，通过修改  Prompt ，模型的关注点倾向了具体特征与技术细节。

我可能进一步想要在描述的结尾展示出产品ID。因此，我可以进一步改进这个  Prompt ，要求在描述的结尾，展示出说明书中的7位产品ID。

In [ ]:
# 更进一步，要求在描述末尾包含 7个字符的产品ID
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our beautiful medieval-style office furniture collection, featuring file cabinets, desks, bookshelves, and conference tables. Available in multiple shell colors and base coatings, with optional plastic or fabric/leather decorations. The chair comes with or without armrests and has a plastic-coated aluminum base with five wheels and pneumatic height adjustment. Suitable for home or commercial use. Made in Italy.

Product IDs: SWC-100, SWC-110


In [20]:
# 更进一步
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

使用最多50个单词。

技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

中世纪风格办公椅，铸铝底座配改性尼龙PA6/PA66涂层（10mm厚），五轮气动升降结构。HD36泡沫座椅可选两种密度，装饰含SWC-100（塑料靠背）或SWC-110（全饰面料/皮革）。底座涂层：不锈钢/哑光黑/光泽白/铬。意大利制造。产品ID: SWC-100, SWC-110.  

（单词数：50）


以上是许多开发人员通常会经历的  Prompt 开发的迭代过程简短示例。我的建议是，像上一章中所演示的那样，Prompt 应该保持清晰和明确，并在必要时给模型一些思考时间。在这些要求的基础上，常见流程是首先尝试编写一版 Prompt ，看看会发生什么，然后继续迭代完善 Prompt，以逐渐接近所需的结果。许多成功的 Prompt 都是通过这种迭代过程得出的。我将向您展示一个更复杂的 Prompt 示例，可能会让您对 ChatGPT 的能力有更深入的了解。

## 2.3 问题三：添加表格描述
继续添加指引，要求提取产品尺寸信息并组织成表格，并指定表格的列、表名和格式；再将所有内容格式化为可以在网页使用的 HTML。

In [ ]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <p>Introducing our beautiful collection of medieval-style office furniture, including file cabinets, desks, bookcases, and conference tables. Choose from a variety of shell colors and base coatings. You can opt for plastic front and backrest decoration (SWC-100) or full decoration with 10 fabrics and 6 leathers (SWC-110). Base coating options include stainless steel, matte black, glossy white, or chrome. The chair is available with or without armrests and is suitable for both home and commercial settings. It is contract eligible.</p>
  <p>The structure features a plastic-coated aluminum base with five wheels. The chair is pneumatically adjustable for easy height adjustment.</p>
  <p>Product IDs: SWC-100, SWC-110</p>
  <table>
    <caption>Product Dimensions</caption>
    <tr>
      <td>Width</td>
      <td>20.87 inches</td>
    </tr>
    <tr>
      <td>Depth</td>
      <td>20.08 inches</td>
    </tr>
    <tr>
      <td>Height</td>
      <td>31.50 inches</td>
    </tr>
    <tr>


In [ ]:
# 表格是以 HTML 格式呈现的，加载出来
from IPython.display import display, HTML

display(HTML(response))

Width,20.87 inches
Depth,20.08 inches
Height,31.50 inches
Seat Height,17.32 inches
Seat Depth,16.14 inches


In [21]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

在描述之后，包括一个表格，提供产品的尺寸。表格应该有两列。第一列包括尺寸的名称。第二列只包括英寸的测量值。

给表格命名为“产品尺寸”。

将所有内容格式化为可用于网站的HTML格式。将描述放在<div>元素中。

技术规格：```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <h2>中世纪风格办公椅 - 专业系列</h2>
  <p>这款专业级办公椅是我们中世纪风格办公家具系列的核心产品，专为商业和高端住宅环境设计。采用模块化结构系统，提供高度定制化配置选项。</p>

  <h3>核心结构与材料</h3>
  <p><strong>底座系统：</strong>五爪式重力铸造铝基座，表面采用改性尼龙PA6/PA66复合涂层工艺。配备工业级气动升降柱，支持250次/日升降循环。可选配软/硬地板专用双轴承聚氨酯滚轮系统。</p>
  
  <p><strong>壳体结构：</strong>10mm厚度强化壳体采用高压注射成型工艺，可选：
  <ul>
    <li>SWC-100型：前后塑料装饰嵌件</li>
    <li>SWC-110型：全包覆式装饰（10种纺织面料/6种全粒面皮革选项）</li>
  </ul></p>

  <p><strong>座椅系统：</strong>高密度HD36冷固化泡沫芯材，可选1.8lb/ft³（中等密度）或2.8lb/ft³（高密度）两种配方。配备8向可调PU扶手系统（可选无扶手配置）。</p>

  <h3>技术规格</h3>
  <p>底座涂层选项：不锈钢/哑光黑/高光白/镀铬<br>
  符合ANSI/BIFMA商业等级标准<br>
  原产地：意大利</p>

  <p>产品ID：SWC-100 | SWC-110</p>
</div>

<table class="product-dimensions">
  <caption>产品尺寸</caption>
  <tr>
    <th>尺寸名称</th>
    <th>英寸</th>
  </tr>
  <tr>
    <td>宽度</td>
    <td>20.87</td>
  </tr>
  <tr>
    <td>深度</td>
    <td>20.08</td>
  </tr>
  <tr>
    <td>高度</td>
    <td>31.50</td>
  </tr>
  <tr>
    <td>座椅高度</td>
    <td>17.32</td>
  </tr>
  <tr>
    <td>座椅深度</td>
    <td>16.14</td>
  </tr>
</table>


In [22]:
# 表格是以 HTML 格式呈现的，加载出来
from IPython.display import display, HTML

display(HTML(response))

尺寸名称,英寸
宽度,20.87
深度,20.08
高度,31.50
座椅高度,17.32
座椅深度,16.14


本章的主要内容是 LLM 在开发应用程序中的迭代式  Prompt 开发过程。开发者需要先尝试编写  Prompt ，然后通过迭代逐步完善它，直至得到需要的结果。作为一名高效的提示词工程师（Prompt Engineer），关键在于掌握有效的开发Prompt的过程，而不是去寻求得到“完美的”Prompt。对于一些更复杂的应用程序，可以对多个样本（如数百张说明书）进行  Prompt 的迭代开发，并在样本集上进行评估。

最后，在更成熟的应用程序中，可以观察多个Prompt在多个样本集上的表现，测试平均或最差性能。但通常，**仅当**应用较成型之后，才推荐您通过这种评估方式，来精益求精。

请使用 Jupyter Notebook，动手实践本节给出的示例，并尝试不同的变化，查看结果。